## Teste dos modelos de classificação
### Random Forest

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings(action = 'ignore')

df = pd.read_csv('Datasets/dataset_comp2.csv', sep = ';')
df

,corrente_cp11,corrente_cp12,corrente_cp13,corrente_cp31,vazaro_ar,pressao_ar,falha
0,96.900002,92.599998,97.000000,97.400002,21204.308594,516.897827,0
1,96.900002,92.500000,97.000000,97.400002,21073.390625,516.804749,0
2,96.900002,92.300003,96.900002,97.800003,21070.710938,517.828857,0
3,96.800003,92.500000,97.000000,97.400002,21106.867188,518.573669,0
4,97.000000,92.599998,97.199997,97.400002,21230.927734,518.759949,0
...,...,...,...,...,...,...,...
61357,99.699997,96.000000,111.000000,98.400002,20611.375000,592.868469,0
61358,99.500000,95.599998,110.699997,97.800003,20561.994141,594.916626,0
61359,99.599998,95.699997,110.900002,98.700005,20605.894531,594.637390,0
61360,99.400002,95.599998,110.699997,98.099998,20581.210938,595.568359,0


In [2]:
# Importando a biblioteca necesária para separação dos dados

from sklearn.model_selection import train_test_split
from sklearn import metrics

# Váriavel Y
y = df['falha']

# Váriavel X
X = df[['corrente_cp11','vazaro_ar', 'pressao_ar']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 2811)

print("Dados separados em treino e teste")

Dados separados em treino e teste


### Modelo Linear Regression

In [3]:
from sklearn.ensemble import RandomForestClassifier

modelo = RandomForestClassifier()
modelo.fit(X_train,y_train)
print("Modelo criado")

Modelo criado


In [4]:
y_pred= modelo.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

#### Teste manual

In [5]:
colunas = X_test.columns.to_list()
entrada = X_test[0:1]
linha = df.iloc[[0]]
linha

,corrente_cp11,corrente_cp12,corrente_cp13,corrente_cp31,vazaro_ar,pressao_ar,falha
0,96.900002,92.599998,97.0,97.400002,21204.308594,516.897827,0


In [6]:
linha_pred = modelo.predict(linha[colunas])
linha_pred

array([0], dtype=int64)

In [7]:
# Teste hipotese

# Assumimos que haverá alerta para valr de corrente acima de 100, vazão menor que 21038
# vamos avaliar se o modelo prever a condição de falha nessa situação

In [8]:
linha_2 = linha.copy()
linha_2['corrente_cp11'] = 101
linha_2['vazaro_ar'] = 21038
print("Previsão Linha 2: {}".format(modelo.predict(linha_2[colunas])))

Previsão Linha 2: [1]


#### Metricas

In [9]:
acuracia = metrics.accuracy_score(y_test, y_pred)
print(acuracia)

1.0


In [10]:
precisao = metrics.precision_score(y_test, y_pred)
print(precisao)

1.0


In [11]:
recall_pred = metrics.recall_score(y_test, y_pred)
print(recall_pred)

1.0


In [12]:
matriz = metrics.confusion_matrix(y_test, y_pred)
print(matriz)

[[19850     0]
 [    0   400]]


##### Melhor modelo encontrado para o problema

In [13]:
import itertools

linhas = []

entrada = linha.copy()
corrente_cp11 = 101

for (i, j) in itertools.product(range(21038,23066), range(504,637)):
    vazaro_ar = i
    pressao_ar = j
    entrada = [[corrente_cp11,vazaro_ar, pressao_ar]]
    previsao_falha = int(modelo.predict(entrada)[0])
    
    linhas.append([previsao_falha, vazaro_ar, pressao_ar])
    
df_pred = pd.DataFrame(linhas, columns = ["falha","vazao","pressao"])
print("Dataset concluído")

Dataset concluído


In [14]:
# Análise do menor valor de vazão e pressão para de corrente em alerta

df_pred.nsmallest(1,'pressao')

,falha,vazao,pressao
0,1,21038,504


In [15]:
df_pred.to_csv('Datasets/dataset_final.csv', sep = ';', index = False)
df_pred.head()

,falha,vazao,pressao
0,1,21038,504
1,1,21038,505
2,1,21038,506
3,1,21038,507
4,1,21038,508
